In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, ArrayType
from pyspark.sql.functions import col                       # Filtering using the col() function
from pyspark.sql.functions import array_contains            # Filtering on array columns
from pyspark.sql.functions import explode                   # Explode Arrays in Individual Rows
from pyspark.sql.functions import sum, avg, count, max      # Multiple Aggregations
from pyspark.sql.functions import first, last
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import lit, array

# Create an entry point to the PySpark Application
spark = SparkSession.builder \
      .master("local") \
      .appName("MyFirstSparkApplication") \
      .getOrCreate()

In [3]:
#IMPORT

df_book = spark.read.option("multiline","true")  \
      .json("Datasets/book-db.json")

df_article = spark.read.option("multiline","true")  \
      .json("Datasets/article-db.json")

df_incollection = spark.read.option("multiline","true")  \
      .json("Datasets/incollection-db.json")

df_www = spark.read.option("multiline","true")  \
      .json("Datasets/www-db.json")


In [ ]:
df_article.show()
df_book.show()
df_incollection.show()
df_www.show()


# **1. Top Journals for heterogeneity of topics**
This query is useful to obtain the journals which are the most heterogeneous, i.e. the ones whose articles cover a good number of topics. 
In this case, we want the top 10 journals that have published articles related to at least 25 different keywords.

In [4]:
result = df_article \
    .select(col("journal"), explode("keyword")) \
    .withColumnRenamed("col", "keyword") \
    .groupBy("journal") \
    .agg(countDistinct("journal", "keyword")) \
    .withColumnRenamed("count(journal, keyword)", "number of keywords") \
    .filter(col("number of keywords") > 25) \
    .sort(col("number of keywords").desc())
    
result.limit(10).show(truncate = False)


+----------------------------+------------------+
|journal                     |number of keywords|
+----------------------------+------------------+
|SIGMOD Rec.                 |45                |
|IEEE Trans. Knowl. Data Eng.|45                |
|IEEE Data Eng. Bull.        |45                |
|ACM SIGMOD Digit. Rev.      |45                |
|IWBS Report                 |45                |
|ACM Trans. Database Syst.   |45                |
|VLDB J.                     |45                |
|LILOG-Report                |44                |
|ACM Comput. Surv.           |44                |
|Commun. ACM                 |28                |
+----------------------------+------------------+



# **2. Find books not treating a certain topic and related to favourite Publishers**
This query is useful if you want to buy a book that has been published by one of your favourite publishers but does not treat a certain topic, for example because you have already studied it or because you just simply don't like it.

In [24]:
favourite_publishers = ["CRC Press", "World Scientific", "Routledge", "SIAM", "Atlantis Press"]
hated_topic = "data processing"
df_book_favPubl = df_book.filter(col("publisher").isin(favourite_publishers))
result = df_book_favPubl.filter(array_contains(df_book_favPubl.keyword, hated_topic) == False) \
    .select(col("isbn"), col("publisher"), col("title"), col("year")) \
    .limit(5) \
    .show(truncate=False)

+--------------------------------------+----------------+-------------------------------------------+----+
|isbn                                  |publisher       |title                                      |year|
+--------------------------------------+----------------+-------------------------------------------+----+
|[978-94-6239-185-7, 978-94-6239-186-4]|Atlantis Press  |Introduction to Text Visualization         |2016|
|[9781138207950]                       |Routledge       |Cognitive Design for Artificial Minds      |2021|
|[978-1-611-97368-6]                   |SIAM            |Evaluating Gas Network Capacities          |2015|
|[978-981-4632-12-6, 978-981-4632-14-0]|World Scientific|Adaptive Cloud Enterprise Architecture     |2015|
|[9781315369228]                       |CRC Press       |Information Theory Tools for Visualization.|2016|
+--------------------------------------+----------------+-------------------------------------------+----+



# **3. Update the available URL for a certain author**
This query could be useful in a scenario where an author decides that the link to his webpage (specified by him to the dataframe admin) should be the only link available among the various URLs associated to him in the dataframe, thus deleting all the other URLs uploaded in the past.

In [21]:
df_www_exploded = df_www  \
    .select(explode(df_www.author), df_www.url)   \
    .withColumnRenamed("col", "author")

result = df_www_exploded \
    .filter(col("author.name") == "Elena Ferrari") \
    .withColumn("url", lit("http://www.dicom.uninsubria.it/~elena.ferrari/")) \
    .select(col("author.name"), col("url")) \
    .show(truncate=False)

+-------------+----------------------------------------------+
|name         |url                                           |
+-------------+----------------------------------------------+
|Elena Ferrari|http://www.dicom.uninsubria.it/~elena.ferrari/|
+-------------+----------------------------------------------+

